In [1]:
import xgboost

print(xgboost.build_info())

{'BUILTIN_PREFETCH_PRESENT': False, 'CUDA_VERSION': [11, 8], 'DEBUG': False, 'MM_PREFETCH_PRESENT': True, 'THRUST_VERSION': [1, 15, 1], 'USE_CUDA': True, 'USE_FEDERATED': False, 'USE_NCCL': False, 'USE_OPENMP': True, 'USE_RMM': False, 'libxgboost': 'c:\\Users\\Paul\\anaconda3\\envs\\dvhope-xgb2\\lib\\site-packages\\xgboost\\lib\\xgboost.dll'}


In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from ray import tune
from ray.tune.search.bayesopt import BayesOptSearch

# Generate a synthetic classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function
def objective(config):
    model = XGBClassifier(
        n_estimators=config["n_estimators"],
        max_depth=config["max_depth"],
        learning_rate=config["learning_rate"],
        subsample=config["subsample"],
        colsample_bytree=config["colsample_bytree"],
        random_state=42,
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return {"accuracy": accuracy}

# Define the search space
config = {
    "n_estimators": tune.randint(50, 500),
    "max_depth": tune.randint(2, 10),
    "learning_rate": tune.loguniform(1e-3, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "colsample_bytree": tune.uniform(0.5, 1.0),
}

# Create a Ray Tune experiment
bayesopt = BayesOptSearch(metric="accuracy", mode="max")

analysis = tune.run(
    objective,
    config=config,
    search_alg=bayesopt,
    num_samples=50,
    resources_per_trial={"cpu": 2},
)

# Get the best hyperparameters and corresponding accuracy
best_config = analysis.get_best_config(metric="accuracy", mode="max")
best_accuracy = analysis.best_result["accuracy"]

print(f"Best hyperparameters: {best_config}")
print(f"Best accuracy: {best_accuracy:.4f}")

2024-11-15 10:48:59,449	INFO worker.py:1642 -- Started a local Ray instance.
2024-11-15 10:49:00,847	INFO tune.py:228 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2024-11-15 10:49:00,848	INFO tune.py:645 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


ValueError: BayesOpt does not support parameters of type `Integer`